In [ ]:
'''
This notebook is to be used to convert best-fit parameter values
saved in climatools.lblnew.bestfit_params into Fortran, in a form
that can be easily inserted into the kdist_params() subroutine in
the overlap version of lblnew.f
'''

In [5]:
import itertools


from climatools.lblnew import bestfit_params as bestfit

In [78]:
def into_chunks(l, chunksize):
    return itertools.zip_longest(*(chunksize * [iter(l)]))



def vector_to_F77list(array, num_values_per_line=4, dtype=float):
    '''
    '''
    if dtype == float:
        strfmt = '{:15.6e}_r8'
    elif dtype == int:
        strfmt = '{:15d}'
    else:
        raise ValueError('dtype must be either float or int.')
    
    chunks = into_chunks(array, num_values_per_line)
    
    chunks = list(chunks)
    
    lines = []
    for chunk in chunks[:-1]:
        vs = [strfmt.format(v) for v in chunk if v != None]
        line = ','.join(vs)
        line = line + ','
        lines.append(line)
        
    vs = [strfmt.format(v) for v in chunks[-1] if v != None]
    line = ','.join(vs)
    lines.append(line)
    
    return lines


def vector_to_F77(array=None, num_values_per_line=None, dtype=None):
    lines = vector_to_F77list(array=array, 
                              num_values_per_line=num_values_per_line,
                              dtype=dtype)
    
    rlines = [5 * ' ' + '&' + l for l in lines]
    
    fortran = '\n'.join(rlines)
    return fortran

In [32]:
param = bestfit.kdist_params(molecule='co2', band='3a')

In [80]:
def if_conditional(param):
    vmin, vmax = param['vmin'], param['vmax']
    molecule, band = param['molecule'], param['band']
    
    s_if = "if (vstar == {vmin}_r8 .and. vend == {vmax}_r8) then"
    
    s_elif = ("else if (vstar == {vmin}_r8 "
              ".and. vend == {vmax}_r8) then")
    
    first_bands = [(molecule == 'h2o' and band == '1'),
                   (molecule == 'co2' and band == '3a'),
                   (molecule == 'o3' and band == '5'),
                   (molecule == 'n2o' and band == '7'),
                   (molecule == 'ch4' and band == '6')]
    
    if any(first_bands):
        s = s_if
    else:
        s = s_elif
        
    return s.format(vmin=vmin, vmax=vmax)


def comment_header(param):
    s = "! {} band{}"
    return s.format(param['molecule'], param['band'])


def nref(param):
    s = "nref = {:d}"
    return s.format(len(param['ng_refs']))


def ng(param):
    s = "ng = {:d}"
    return s.format(sum(param['ng_refs']))


def ng_refs(param):
    s = vector_to_F77(param['ng_refs'], num_values_per_line=2, dtype=int)
    s = 



In [81]:
print(ng_refs(param))

['              3,              2,', '              4']


In [35]:
param

{'band': '3a',
 'klin': 6.375563e-24,
 'molecule': 'co2',
 'ng_adju': [0, 0, 0],
 'ng_refs': [3, 2, 4],
 'option_wgt_k': 1,
 'ref_pts': [(1, 250), (10, 250), (500, 250)],
 'vmax': 620,
 'vmin': 540,
 'w_diffuse': [(1.6, 1.6, 1.7), (1.75, 1.75), (1.55, 1.55, 1.6, 1.85)],
 'wgt': [(0.7, 0.3, 0.7), (0.7, 0.6), (0.4, 0.5, 0.8, 0.95)]}

In [22]:
def gasband_str_funcs():
    return (if_conditional,
            comment_header,
            nref,
            ng,
            ng_refs,
            ng_adju,
            p_refs,
            t_refs,
            wgt,
            w_diffuse,
            option_klin,
            klin)


In [34]:
def kdist_param_gasband(param):
    
    for i, f in enumerate(gasband_str_funcs()):
        if i == 0:
            lines.append(6 * ' ' + f(param))
        else:
            lines.append(6 * ' ' + 3 * ' ' + f(param))
            
    return lines
        